### Ingest circuits.csv file

#### Step 1 - Read the CSV file using the spark dataframe reader

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")
v_data_source

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
circuits_schema = StructType(fields = [
    StructField("circuitId", IntegerType(), False),
    StructField("circuitRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("country", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lng", DoubleType(), True),
    StructField("alt", IntegerType(), True),
    StructField("url", StringType(), True)
])

In [0]:
# inferSchema will add an additional spark job and will slow down the process so it is not recommended in production
circuits_df = (
    spark.read
    .option("header", True)
    .schema(circuits_schema)
    .csv(f"{raw_folder_path}/circuits.csv")
)

In [0]:
display(circuits_df)

In [0]:
circuits_df.printSchema()

In [0]:
circuits_df.describe().show()


#### Select only the reqired columns

In [0]:
from pyspark.sql.functions import col

circuits_selected_df = circuits_df.select("circuitId", "circuitRef", "name", "location", "country", "lat", "lng", "alt")

# circuits_selected_df = circuits_df.select(circuits_df.circuitId, circuits_df.circuitRef, circuits_df.name, circuits_df.location, circuits_df.country, circuits_df.lat, circuits_df.lng, circuits_df.alt)

# circuits_selected_df = circuits_df.select(circuits_df["circuitId"], circuits_df["circuitRef"], circuits_df["name"], circuits_df["location"], circuits_df["country"], circuits_df["lat"], circuits_df["lng"], circuits_df["alt"])

# circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country").alias("race_country"), col("lat"), col("lng"), col("alt"))

display(circuits_selected_df)

#### Rename columns

In [0]:
from pyspark.sql.functions import lit

circuits_renamed_df = circuits_selected_df \
    .withColumnRenamed("circuitId", "circuit_id") \
    .withColumnRenamed("circuitRef", "circuit_ref") \
    .withColumnRenamed("lat", "latitude") \
    .withColumnRenamed("lng", "longitude") \
    .withColumnRenamed("alt", "altitude") \
    .withColumn("data_source", lit(v_data_source))

display(circuits_renamed_df)

### Add ingestion date

In [0]:

circuits_final_df = add_ingestion_date(circuits_renamed_df)

display(circuits_final_df)

### Write data to datalake as parquet

In [0]:
circuits_final_df.write.mode('overwrite').format("parquet").saveAsTable("f1_processed.circuits")

In [0]:
%fs
ls /mnt

In [0]:
df = spark.read.parquet(f"{processed_folder_path}/circuits")

In [0]:
display(df)

In [0]:
dbutils.notebook.exit("Success")